## YouTube Video Analysis on Vertex AI using PaLM and LangChain

In [ ]:
!pip install --upgrade --user google-cloud-aiplatform>=1.29.0 google-cloud-storage langchain pytube youtube-transcript-api chromadb gradio pydantic
# You may safely ignore dependency warnings below

In [3]:
# restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)


{'status': 'ok', 'restart': True}

In [1]:
# flist = !ls

In [2]:
# get project ID
PROJECT_ID = ! gcloud config get project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # <-- Enter assigned region here.

# generate an unique id for this session
from datetime import datetime
UID = datetime.now().strftime("%m%d%H%M")

In [ ]:
# init the vertexai package
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [44]:
from langchain.document_loaders import YoutubeLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.llms import VertexAI


### Load PALM (bison) LLM model

In [5]:

llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)



In [10]:
# import langchain.llms as LLMS_lc
# help(LLMS_lc)
# help(VertexAI)

### Load pretrained text embeddings

In [11]:
from langchain.embeddings import VertexAIEmbeddings

# Embedding
EMBEDDING_QPM = 100
EMBEDDING_NUM_BATCH =5

embeddings = VertexAIEmbeddings(
    requests_per_minute=EMBEDDING_QPM,
    num_instances_per_batch=EMBEDDING_NUM_BATCH,
)

Model_name will become a required arg for VertexAIEmbeddings starting from Feb-01-2024. Currently the default is set to textembedding-gecko@001


### Load YouTube video Transcripts

In [16]:
loader = YoutubeLoader.from_youtube_url("https://www.youtube.com/watch?v=XX2XpqklUrE", add_video_info=True)
result = loader.load()


In [50]:
# display video transcript 
result

### Split transcripts into chunks

In [18]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=0)
docs = text_splitter.split_documents(result)
print(f"# of documents = {len(docs)}")


# of documents = 4


### Create embeddings for  video-transcript documents using pretrained embeddings 

In [36]:
embedding_list = embeddings.embed_documents([doc.page_content for doc in docs])
print(f"You have {len(embedding_list)} embeddings")
print(f"Here's a sample of one: {embedding_list[0][:3]}...")


You have 4 embeddings
Here's a sample of one: [-0.03151746094226837, -0.042533669620752335, 0.004869665950536728]...


### Index video-transcript as Document embeddings using a vector database here we are using Chroma DB

In [42]:
db = Chroma.from_documents(docs, embeddings)
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 2}) # here search_kwargs how many top similar documents to return


In [46]:
# help(db.as_retriever)

### Initialize a Question answering langchain application with palm llm and context info. (via index to vector db)  

- here chain_type = 'stuff' indicates how we need to pass the data into the prompt for the llm. i.e. stuff the context data into prompt.

In [47]:
qa = RetrievalQA.from_chain_type( llm=llm, chain_type="stuff", retriever=retriever, return_source_documents=True)


In [48]:
def sm_ask(question, print_results=True):
  video_subset = qa({"query": question})
  context = video_subset
  prompt = f"""
  Answer the following question in a detailed manner, using information from the text below. If the answer is not in the text,say I dont know and do not generate your own response.

  Question:
  {question}
  Text:
  {context}

  Question:
  {question}

  Answer:
  """
  parameters = {
      "temperature": 0.1,
      "max_output_tokens": 256,
      "top_p": 0.8,
      "top_k": 40
  }
  response = llm.predict(prompt, **parameters)
  return {
      "answer": response
  }


### Using gradio we generate a simple UI for our Question Answering application.

In [49]:
import gradio as gr
def get_response(input_text):
  response = sm_ask(input_text)
  return response

grapp = gr.Interface(fn=get_response, inputs="text", outputs="text")
grapp.launch(share=True)


Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://009edab91dd5a3b17e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### sample questions we can use to test the above:

- What is this video about and who are the speakers?
- How is the tone and sentiment of this video?
- Who are the target audience for this video?
- Is this a marketing video by Google Cloud?